<a href="https://colab.research.google.com/github/jayw20230711/COLAB/blob/main/Insurance_Chatbot_cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# %% [markdown]
"""
# Insurance Customer Service Chatbot (RAG System)
Build a knowledge-grounded chatbot for insurance queries using Retrieval-Augmented Generation*
"""
# %%
!pip install -q langchain openai faiss-cpu pypdf sentence-transformers streamlit tiktoken
# %%
from IPython.display import HTML, display
import sys

def set_up_colab():
    display(HTML('''
    <script>
      function startStreamlit() {
        var button = document.querySelector("button");
        button.disabled = true;
        button.textContent = "Running Streamlit...";

        google.colab.kernel.proxyPort(8501, {
          'cache': false,
          'onBackendReady': function() {
            var iframe = document.createElement('iframe');
            iframe.src = 'https://localhost:8501';
            iframe.width = '100%';
            iframe.height = '600px';
            iframe.frameBorder = 0;

            var output = document.getElementById('output');
            output.innerHTML = '';
            output.appendChild(iframe);
          }
        });
      }
    </script>
    <button onclick="startStreamlit()">Launch Chatbot UI</button>
    <div id="output"></div>
    '''))
# %% [markdown]

In [8]:
pip install -U langchain-community

In [9]:
"""
## Step 1: Set Up Knowledge Base
We'll use sample insurance documents (you should replace with your actual data)
"""
# %%
!mkdir -p sample_data
!echo "Home Insurance Policy\nCoverage includes:\n- Fire damage\n- Theft\n- Natural disasters (excluding floods)\nDeductible: $500" > sample_data/home_policy.txt
!echo "Claims Process:\n1. Report incident within 72 hours\n2. Submit claim form\n3. Provide evidence (photos, police report)\n4. Wait 5-7 business days for processing" > sample_data/claims_process.txt
# %%
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader('sample_data/', glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len
)
chunks = text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(chunks)} chunks")
print("Sample chunk:", chunks[0].page_content[:100] + "...")
# %% [markdown]


Split 2 documents into 2 chunks
Sample chunk: Claims Process:\n1. Report incident within 72 hours\n2. Submit claim form\n3. Provide evidence (phot...


In [10]:
!pip install -U langchain-cohere

In [11]:
from langchain_cohere import CohereEmbeddings

In [12]:
"""
## Step 2: Create Vector Database
We'll use FAISS for efficient similarity search
"""
# %%
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
import os
import cohere
from google.colab import userdata

# Get your Cohere API key: https://dashboard.cohere.com/api-keys

# Get the secret from Colab's Secrets manager
cohere_api_key = userdata.get('COHERE_API_KEY')

# Set it as an environment variable
os.environ['COHERE_API_KEY'] = cohere_api_key

# Verify that the variable is set (optional)
print(os.environ['COHERE_API_KEY'])

# Use SentenceTransformerEmbeddings as CohereEmbeddings is causing issues
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print("Using SentenceTransformerEmbeddings.")

vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("insurance_vector_db")
# %% [markdown]



KKQPLS80RNyVyFAp6ztFn0ReLhGeGOjDEazz0YNn


/tmp/ipython-input-344526459.py:24: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Using SentenceTransformerEmbeddings.


In [13]:
"""
## Step 3: Implement the RAG Chain
Now we'll set up the Retrieval-Augmented Generation chain.
"""
# %%
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQA

# Initialize a Cohere chat model
llm = ChatCohere(cohere_api_key=cohere_api_key)
print("Initialized Cohere chat model.")

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever()
)
print("Created RetrievalQA chain.")

Initialized Cohere chat model.
Created RetrievalQA chain.


# Task
Create a chatbot using Langchain and Cohere to answer questions about insurance documents.

## Set up the chatbot interface (streamlit app)

### Subtask:
Create a Python script for the Streamlit application that will serve as the user interface for the chatbot.


**Reasoning**:
The subtask is to create a Streamlit application file. This requires writing the code for the Streamlit UI in a new Python file. The code should include the basic structure for a chat interface: setting the title, initializing and displaying chat history, and creating a user input field.



In [14]:
%%writefile app.py
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage

st.title("Insurance Chatbot Based on Cohere API")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    if isinstance(message, HumanMessage):
        with st.chat_message("user"):
            st.markdown(message.content)
    elif isinstance(message, AIMessage):
        with st.chat_message("assistant"):
            st.markdown(message.content)

# Accept user input
if prompt := st.chat_input("Ask a question about insurance policies or claims:"):
    # Add user message to chat history
    st.session_state.messages.append(HumanMessage(content=prompt))
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response (placeholder for now)
    with st.chat_message("assistant"):
        # This is a placeholder. The actual response will be generated by the RAG chain.
        st.markdown("Thinking...")

Writing app.py


## Implement the rag chain

### Subtask:
Define the logic for the Retrieval-Augmented Generation (RAG) system using Langchain components, including retrieving relevant documents from the vector database and generating a response using a language model.


**Reasoning**:
The previous step failed due to a KeyError related to 'user_agent' when initializing the CohereEmbeddings. This seems to be an issue with the latest version of the Cohere library or its integration with Langchain. I will try to initialize the embeddings without specifying a model explicitly, or use a different embedding model if necessary, and then load the vector database. After successfully loading the vector database, I will initialize the Cohere chat model and create the RetrievalQA chain as per the instructions.



In [15]:
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQA
import os

# Get your Cohere API key: https://dashboard.cohere.com/api-keys
# Using the API key set in the previous cell
cohere_api_key = os.environ['COHERE_API_KEY']

# Initialize CohereEmbeddings - trying without explicit model name first
try:
    embeddings = CohereEmbeddings()
except Exception as e:
    print(f"Error initializing CohereEmbeddings without model: {e}")
    # If that fails, try with the model name again, maybe the error is intermittent or related to another issue
    try:
        embeddings = CohereEmbeddings(model="embed-english-v2.0")
        print("Successfully initialized CohereEmbeddings with model 'embed-english-v2.0'")
    except Exception as e:
        print(f"Error initializing CohereEmbeddings with model 'embed-english-v2.0': {e}")
        # As a fallback, use a different embedding model if Cohere fails
        from langchain_community.embeddings import SentenceTransformerEmbeddings
        embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
        print("Using SentenceTransformerEmbeddings as fallback.")

# Load the saved FAISS vector database
vector_db = FAISS.load_local("insurance_vector_db", embeddings, allow_dangerous_deserialization=True)
print("Loaded FAISS vector database.")

# Initialize a Cohere chat model
llm = ChatCohere(cohere_api_key=cohere_api_key)
print("Initialized Cohere chat model.")

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever()
)
print("Created RetrievalQA chain.")

Error initializing CohereEmbeddings without model: 'user_agent'
Error initializing CohereEmbeddings with model 'embed-english-v2.0': 'user_agent'


/tmp/ipython-input-129711054.py:13: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-cohere package and should be used instead. To use it run `pip install -U :class:`~langchain-cohere` and import as `from :class:`~langchain_cohere import CohereEmbeddings``.
  embeddings = CohereEmbeddings()


Using SentenceTransformerEmbeddings as fallback.
Loaded FAISS vector database.
Initialized Cohere chat model.
Created RetrievalQA chain.


**Reasoning**:
The previous attempt failed because CohereEmbeddings could not be initialized due to a 'user_agent' KeyError, and subsequently, loading the FAISS database failed because it was never successfully saved in the first place due to the embedding initialization issue. Since CohereEmbeddings is not working, I will use SentenceTransformerEmbeddings (which successfully initialized in the previous attempt) to create and save the FAISS vector database. Then, I will proceed with initializing the Cohere chat model and creating the RetrievalQA chain as originally planned.



In [16]:
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_cohere import ChatCohere
from langchain.chains import RetrievalQA
import os

# Get your Cohere API key
cohere_api_key = os.environ['COHERE_API_KEY']

# Use SentenceTransformerEmbeddings as CohereEmbeddings failed
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
print("Using SentenceTransformerEmbeddings to create the vector database.")

# Create and save the FAISS vector database using the fallback embeddings
vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("insurance_vector_db")
print("Created and saved FAISS vector database.")

# Initialize a Cohere chat model
llm = ChatCohere(cohere_api_key=cohere_api_key)
print("Initialized Cohere chat model.")

# Create a RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_db.as_retriever()
)
print("Created RetrievalQA chain.")

Using SentenceTransformerEmbeddings to create the vector database.
Created and saved FAISS vector database.
Initialized Cohere chat model.
Created RetrievalQA chain.


## Integrate chat history

### Subtask:
Add functionality to the RAG chain to handle chat history, allowing for more conversational interactions.


**Reasoning**:
Import the necessary components for managing chat history and conversational retrieval chains from `langchain.chains`. Define a new conversational retrieval chain using `ConversationalRetrievalChain.from_llm` with the existing language model and retriever.



In [17]:
from langchain.chains import ConversationalRetrievalChain

# Define a new conversational retrieval chain
# This chain takes into account chat history
conversational_qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever()
)

print("Defined conversational retrieval chain.")

Defined conversational retrieval chain.


**Reasoning**:
Update the `app.py` file to utilize the new conversational retrieval chain for generating responses and modify the response generation part to handle the output and append it to the chat history.



In [18]:
%%writefile app.py
import streamlit as st
from langchain_core.messages import HumanMessage, AIMessage
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_cohere import ChatCohere
from langchain.chains import ConversationalRetrievalChain
import os

# Get your Cohere API key
# Make sure this is set in your environment or replace with a secure method
cohere_api_key = os.environ.get('COHERE_API_KEY')

# Load the vector database and embeddings
# Using the same embeddings as when the DB was created
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.load_local("insurance_vector_db", embeddings, allow_dangerous_deserialization=True)

# Initialize a Cohere chat model
llm = ChatCohere(cohere_api_key=cohere_api_key)

# Create a conversational retrieval chain
conversational_qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vector_db.as_retriever()
)


st.title("Insurance Chatbot Based on Cohere API")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    if isinstance(message, HumanMessage):
        with st.chat_message("user"):
            st.markdown(message.content)
    elif isinstance(message, AIMessage):
        with st.chat_message("assistant"):
            st.markdown(message.content)

# Accept user input
if prompt := st.chat_input("Ask a question about insurance policies or claims:"):
    # Add user message to chat history
    st.session_state.messages.append(HumanMessage(content=prompt))
    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Generate assistant response using the conversational chain
    with st.chat_message("assistant"):
        response = conversational_qa_chain.invoke({
            "question": prompt,
            "chat_history": st.session_state.messages
        })
        ai_message = AIMessage(content=response["answer"])
        st.session_state.messages.append(ai_message)
        st.markdown(ai_message.content)


Overwriting app.py


## Run the streamlit app

### Subtask:
Execute the Streamlit script to launch the chatbot interface in the Colab output.


**Reasoning**:
Execute the set_up_colab function and then run the streamlit application.



In [ ]:
set_up_colab()
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.64.200:8501

⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) 

In [ ]:
!pip install -U langchain-huggingface